In [53]:
import pandas as pd
import scanpy as sc
import numpy as np

from scarches.models.scpoli import scPoli

In [9]:
scpoli_model_path = "/storage/data/data/organoid_atlas/230510_scpoli_hierarchical123"
ref_adata_path = "/storage/data/data/organoid_atlas/230510_09_organoids_cleaned_hvg.h5ad"
query_adata_path = "/storage/data/data/organoid_atlas/230605_pasca_all_01.h5ad"

In [3]:
ref_adata = sc.read_h5ad(ref_adata_path)
ref_adata.X = ref_adata.layers["counts_lengthnorm"].A.copy()

In [5]:
scpoli_model = scPoli.load(scpoli_model_path, ref_adata)

AnnData object with n_obs × n_vars = 1770578 × 3000
    obs: 'assay_sc', 'assay_differentiation', 'assay_type_differentiation', 'bio_sample', 'cell_line', 'cell_type', 'development_stage', 'disease', 'ethnicity', 'gm', 'id', 'individual', 'organ', 'organism', 'sex', 'state_exact', 'sample_source', 'source_doi', 'suspension_type_original', 'tech_sample', 'treatment', 'assay_sc_original', 'cell_line_original', 'cell_type_original', 'development_stage_original', 'disease_original', 'ethnicity_original', 'organ_original', 'organism_original', 'sex_original', 'suspension_type', 'obs_names_original', 'organoid_age_days', 'publication', 'doi', 'batch', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'leiden_pca_unintegrated_1', 'leiden_pca_unintegrated_80', 'leiden_pca_rss_1', 'leiden_pca_rss_80', 'snapseed_pca_unintegrated_level_1', 'snapseed_pca_unintegrated_level_2', 'snapseed_pca_unintegrate

In [10]:
query_adata = sc.read_h5ad(query_adata_path)

In [19]:
query_adata.obs["batch"] = query_adata.obs["dataset"]

In [31]:
query_adata.obs["snapseed_pca_rss_level_1"] = "unknown"
query_adata.obs["snapseed_pca_rss_level_12"] = "unknown"
query_adata.obs["snapseed_pca_rss_level_123"] = "unknown"

In [20]:
query_adata.X = query_adata.layers["counts"].A.copy()

In [38]:
query_adata = query_adata[:, ref_adata.var.index]

In [39]:
scpoli_query = scPoli.load_query_data(
    adata=query_adata,
    reference_model=scpoli_model,
    labeled_indices=[]
)

/storage/data/artur/repos/scarches/scarches/models/scpoli/scpoli_model.py:766: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['conditions_combined'] = adata.obs[condition_keys].apply(lambda x: '_'.join(x))


Embedding dictionary:
 	Num conditions: [399]
 	Embedding dim: [5]
Encoder Architecture:
	Input Layer in, out and cond: 3000 1024 5
	Mean/Var Layer in/out: 1024 10
Decoder Architecture:
	First Layer in, out and cond:  10 1024 5
	Output Layer in/out:  1024 3000 



In [46]:
scpoli_query.train(
    n_epochs=500,
    pretraining_epochs=500,
    eta=10,
    unlabeled_prototype_training=False
)

The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels are: {'unknown'}
Therefore integer value of those labels is set to -1
The missing labels ar

In [47]:
query_data_latent = scpoli_query.get_latent(
    query_adata,
    mean=True
)

In [48]:
ref_data_latent = scpoli_model.get_latent(
    ref_adata,
    mean=True
)

In [55]:
joint_latent = np.concatenate((ref_data_latent, query_data_latent), axis=0)

In [59]:
# create new adata
joint_adata = sc.AnnData(
    X=joint_latent,
    obs=pd.concat([ref_adata.obs, query_adata.obs], axis=0)
)

In [61]:
joint_adata.obs["query"] = False

In [71]:
indices = range(-len(query_data_latent), 0)
joint_adata.obs.loc[joint_adata.obs.iloc[indices].index, "query"] = True

In [75]:
sc.pp.neighbors(joint_adata)
sc.tl.umap(joint_adata)

In [85]:
joint_adata.obs["query"] = joint_adata.obs["query"].astype(str)

In [89]:
joint_adata.obs["conditions_combined"] = joint_adata.obs["conditions_combined"].astype(str)

In [90]:
joint_adata.write_h5ad("/storage/data/data/organoid_atlas/scpoli_scarches_pasca.h5ad")

In [96]:
sc.pl.umap(
    joint_adata,
    color='snapseed_pca_rss_level_1',
    show=False,
    frameon=False,
    save="_scanpseed.png"
)

/home/ubuntu/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


<Axes: title={'center': 'snapseed_pca_rss_level_1'}, xlabel='UMAP1', ylabel='UMAP2'>

In [97]:
sc.pl.umap(
    joint_adata,
    color='snapseed_pca_rss_level_12',
    show=False,
    frameon=False,
    save="_scanpseed_level12.png"
)

/home/ubuntu/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


<Axes: title={'center': 'snapseed_pca_rss_level_12'}, xlabel='UMAP1', ylabel='UMAP2'>

In [98]:
sc.pl.umap(
    joint_adata,
    color='snapseed_pca_rss_level_123',
    show=False,
    frameon=False,
    save="_scanpseed_level123.png"
)

/home/ubuntu/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


<Axes: title={'center': 'snapseed_pca_rss_level_123'}, xlabel='UMAP1', ylabel='UMAP2'>

In [99]:
sc.pl.umap(
    joint_adata,
    color='class',
    show=False,
    frameon=False,
    save="_pasca_class.png"
)

/home/ubuntu/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


<Axes: title={'center': 'class'}, xlabel='UMAP1', ylabel='UMAP2'>

In [100]:
sc.pl.umap(
    joint_adata,
    color='class2',
    show=False,
    frameon=False,
    save="_pasca_class2.png"
)

/home/ubuntu/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


<Axes: title={'center': 'class2'}, xlabel='UMAP1', ylabel='UMAP2'>

In [101]:
sc.pl.umap(
    joint_adata,
    color='region',
    show=False,
    frameon=False,
    save="_pasca_region.png"
)

/home/ubuntu/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/plotting/_tools/scatterplots.py:392: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(


<Axes: title={'center': 'region'}, xlabel='UMAP1', ylabel='UMAP2'>